In [1]:
# Clone the repo
!git clone https://github.com/as-ideas/DeepForcedAligner

Cloning into 'DeepForcedAligner'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 367 (delta 40), reused 44 (delta 16), pack-reused 290
Receiving objects: 100% (367/367), 3.03 MiB | 19.06 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [2]:
# Install requirements
%cd DeepForcedAligner/  
!pip install -r requirements.txt

/content/DeepForcedAligner
     |████████████████████████████████| 184kB 6.4MB/s 
     |████████████████████████████████| 276kB 12.7MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-cp36-none-any.whl size=201376 sha256=ca0edeee75f8e9a729baad3a09361b96c2b026b85a9cd01fe745b3808cadb640
  Stored in directory: /root/.cache/pip/wheels/ee/10/1e/382bb4369e189938d5c02e06d10c651817da8d485bfd1647c9
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=de76f05784346de59f1b5213063a8cfbbdbe8cc1cb733981afc577c5f16c65a3
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built librosa PyYAML
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Download and extract data (this may take a while)
!wget http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf LJSpeech-1.1.tar.bz2

--2020-10-30 12:03:20--  http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz  53%[=========>          ]   1.38G  46.9MB/s    eta 27s    

In [ ]:
# Update config with paths and settings for speedup
from dfa.utils import read_config, save_config

config = read_config('config.yaml')
config['paths']['dataset_dir'] = 'LJSpeech-1.1'
config['paths']['metadata_path'] = 'LJSpeech-1.1/metadata.csv'
config['training']['epochs'] = 4         # for speedup
config['durations']['method'] = 'beam'   # for speedup
save_config(config, 'config.yaml')

# Preprocess data (tokenize text and convert wavs to mels)
!python preprocess.py --num_workers 2

In [ ]:
# Start tensorboard
%load_ext tensorboard
%tensorboard --logdir dfa_checkpoints

In [ ]:
# Train speech-to-text model
!python train.py

In [ ]:
# Load latest model and extract char durations
!python extract_durations.py --num_workers 2

In [ ]:
# Load and print example durations in mel steps and milliseconds
import numpy as np

durations = np.load('output/durations/LJ001-0002.npy')
mel_step_ms = 1000. * config['audio']['hop_length'] / config['audio']['sample_rate']
text = 'in being comparatively modern.' 

print('ind    char     dur    dur in ms')
for i, (t, dur) in enumerate(zip(text, durations)):
  print(f'{i:#2}      {t}       {dur:#2}       {dur * mel_step_ms:#.4}')


In [ ]:
# For comparison listen to the wav
import librosa
import IPython.display as ipd

sample_rate = config['audio']['sample_rate']
hop_len = config['audio']['hop_length']
wav, _ = librosa.load('LJSpeech-1.1/wavs/LJ001-0002.wav', sr=sample_rate)
ipd.Audio(wav, rate=sample_rate)


In [ ]:
# Cut word out of wav
print(text[9:22])
char_time = np.cumsum(np.pad(durations, (1, 0))) * hop_len
wav_start, wav_end = char_time[9], char_time[21]
wav_cut = wav[wav_start: wav_end]
ipd.Audio(wav_cut, rate=sample_rate)